<div style="float:left;font-size:20px;">
    <h1>SQL Basics</h1>
</div><div style="float:right;"><img src="../assets/banner.jpg"></div>

<hr>

# SQL

Some good example SQL code in:    P:\SQL\HomeDB

In [ ]:
SELECT radius, t_eff FROM Star
  WHERE radius > 1;

In [ ]:
SELECT kepler_id, t_eff FROM Star
  WHERE t_eff >= 5000 and t_eff <= 6000;

In [ ]:
-- Shows what columns and datatypes are for a table
\d Planet;

In [ ]:
# Aggregate functions
COUNT, MIN, MAX, SUM, AVG, STDDEV

SELECT COUNT(*);

# Pick 5 small planets
SELECT koi_name, radius FROM Planet 
ORDER BY radius DESC
LIMIT 5;

In [ ]:
SELECT MIN(radius), MAX(radius), AVG(radius), STDDEV(radius) FROM Planet
  WHERE kepler_name is NULL;

In [ ]:
SELECT kepler_id, COUNT(*) FROM Planet
  GROUP BY kepler_id
  HAVING COUNT(koi_name) > 1
  ORDER BY COUNT(*) DESC;

### Joins

<table1> LEFT OUTER JOIN <table2>
Here all rows from <table1> are kept and missing matches from <table2> are replaced with NULL values.

<table1> RIGHT OUTER JOIN <table2>
All rows from <table2> are kept and missing matches from <table1> are replaced with NULL values.

<table1> FULL OUTER JOIN <table2>
All rows from both tables are kept.

In [ ]:
# Join stars and planets on their kepler radius and require a star-planet radius ratio greater than sun-earth
SELECT s.radius AS sun_radius, p.radius AS planet_radius FROM Star as s
  INNER JOIN Planet as p ON s.kepler_id = p.kepler_id
  WHERE s.radius/p.radius > 1
  ORDER BY s.radius DESC

In [ ]:
SELECT s.radius, COUNT(*) FROM Planet as p
  INNER JOIN Star as s ON p.kepler_id = s.kepler_id
  WHERE s.radius >= 1
  GROUP BY s.kepler_id
  HAVING COUNT(p.koi_name) > 1
  ORDER BY s.radius DESC

To practise your outer joins, write a query which returns the kepler_id, t_eff and radius for all stars in the Star table which haven't got a planet as join partner. Order the resulting table based on the t_eff attribute in descending order.

In [ ]:
SELECT s.kepler_id, s.t_eff, s.radius FROM Star as s
  LEFT OUTER JOIN Planet as p ON s.kepler_id = p.kepler_id
  WHERE p.koi_name IS NULL
  ORDER BY t_eff DESC

### Subqueries

_Co-related_: the subquery is executed for each element of the outer query.

_Non-co-related_: the subquery is executed only once.
    
    If an attribute in the encapsulating query is used in the nested query, it's going to be a co-related subquery.

In [ ]:
SELECT * FROM Star
WHERE Star.radius > (
  SELECT AVG(radius) FROM Star
);

In [ ]:
SELECT ROUND(AVG(p.t_eq), 1), MIN(s.t_eff), MAX(s.t_eff) FROM Planet as p
  INNER JOIN Star as s USING(kepler_id)
  WHERE s.t_eff > (SELECT AVG(t_eff) FROM Star)

### Timing

In [ ]:
\timing
-- Join with subqueries
SELECT s.kepler_id 
FROM Star s
WHERE s.kepler_id IN (
  SELECT p.kepler_id FROM Planet p
  WHERE p.radius < 1
);

-- Join with JOIN operator
SELECT DISTINCT(s.kepler_id)
FROM Star s
JOIN Planet p USING (kepler_id)
WHERE p.radius < 1;

### Setting up a database

Manipulating table content:
    
INSERT: inserts a new row into a table;

DELETE: deletes a specified row;

UPDATE: changes attributes within a row.

In [ ]:
INSERT INTO Star (kepler_id, t_eff, radius)
VALUES (7115384, 3789, 27.384), (8106973, 5810, 0.811), 
      (9391817, 6200, 0.958)

In [ ]:
DELETE FROM Planet
WHERE kepler_id = 2713049;

In [ ]:
UPDATE Star
SET t_eff = 6000
WHERE kepler_id = 2713049;

In [ ]:
DELETE FROM Planet 
  WHERE radius < 0;
  
UPDATE Planet
  SET kepler_name = NULL
  WHERE status <> 'CONFIRMED';

### Create table

`CREATE TABLE <tablename> (
  <attribute1>  <type1>(size1) <constraint1>,
  <attribute2>  <type2>(size2) <constraint2>,
  ...
);`

|Data type | Description|
|----------|------------|
|SMALLINT | Signed two-byte integer|
|INTEGER  | Signed four-byte integer|
|FLOAT|Eight-byte floating-point number|
|CHAR(n)|Fixed-length string with n characters|
|VARCHAR(n)|Variable-length string with maximum n characters|

|Constraint type|Description|
|---------------|-----------|
|NOT NULL|Value cannot be NULL|
|UNIQUE|Value must be unique in the table|
|DEFAULT|Specifies a default if the field is left blank|
|CHECK|Ensures that the value meets a specific condition|
|PRIMARY KEY|Combination of NOT NULL and UNIQUE|
|FOREIGN KEY|Ensures the data matches the specified attribute in another table|

### Create table

`CREATE TABLE Star (
  kepler_id INTEGER,
  t_eff INTEGER,
  radius FLOAT
);`

`INSERT INTO Star VALUES
  (10341777, 6302, 0.815);`
 
#### With constraints
`CREATE TABLE Star (
  kepler_id INTEGER CHECK(kepler_id > 10)
);`

### Example
`CREATE TABLE Planet (
  kepler_id INTEGER NOT NULL,
  koi_name  VARCHAR(15) UNIQUE NOT NULL,
  kepler_name VARCHAR(15),
  status VARCHAR(20) NOT NULL,
  radius FLOAT NOT NULL
);`

`INSERT INTO Planet VALUES
  (6862328,	  'K00865.01',	NULL,           'CANDIDATE', 119.021),
  (10187017,	'K00082.05',	'Kepler-102 b',	'CONFIRMED', 5.286),
  (10187017,	'K00082.04',	'Kepler-102 c',	'CONFIRMED', 7.071);`

### Keys

#### Primary key
`CREATE TABLE Star (
  kepler_id INTEGER PRIMARY KEY 
);`
  
#### Foreign key
`CREATE TABLE Planet (
  kepler_id INTEGER REFERENCES Star (kepler_id)
);`
  
`INSERT INTO Star VALUES (10341777);
INSERT INTO Planet VALUES (10341777);`

#### Insert from CSV

`CREATE TABLE Star (
  kepler_id INTEGER PRIMARY KEY,
  t_eff INTEGER,
  radius FLOAT
);`

`COPY Star (kepler_id, t_eff, radius) 
  FROM 'stars.csv' CSV;`

`SELECT * FROM Star;`

#### Example

`CREATE TABLE Star (
  kepler_id INTEGER PRIMARY KEY,
  t_eff INTEGER NOT NULL,
  radius FLOAT NOT NULL
);`

`CREATE TABLE Planet (
  kepler_id INTEGER REFERENCES Star (kepler_id),
  koi_name  VARCHAR(20) PRIMARY KEY,
  kepler_name VARCHAR(20),
  status VARCHAR(20) NOT NULL,
  period FLOAT,
  radius FLOAT,
  t_eq INTEGER
);`

`COPY Star (kepler_id, t_eff, radius) 
  FROM 'stars.csv' CSV;`

`COPY Planet (kepler_id, koi_name, kepler_name, status, period, radius, t_eq) 
  FROM 'planets.csv' CSV;`

#### ALTER

`SELECT * FROM Star LIMIT 1;`

`ALTER TABLE Star
ADD COLUMN ra FLOAT,
ADD COLUMN decl FLOAT;`
 
`SELECT * FROM Star LIMIT 1;`

`ALTER TABLE Star
DROP COLUMN ra, 
DROP COLUMN decl;`
 
`ALTER TABLE Star
 ALTER COLUMN t_eff SET DATA TYPE FLOAT;`
 
`ALTER TABLE Star
  ADD CONSTRAINT radius CHECK(radius > 0);`

#### Example

`DELETE FROM Star;`

`ALTER TABLE Star
ADD COLUMN ra FLOAT,
ADD COLUMN decl FLOAT;`

`COPY Star (kepler_id, t_eff, radius, ra, decl)
  FROM 'stars_full.csv' CSV;`

### Python + PostgreSQL

In [ ]:
import psycopg2

# Establish the connection
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()

# Execute an SQL query and receive the output
cursor.execute('SELECT 2 + 3;')
records = cursor.fetchall()

print(records)

In [ ]:
### Example

import psycopg2

def select_all(table):

  conn = psycopg2.connect('dbname=db user=grok')
  cursor = conn.cursor()

  query = 'SELECT * FROM ' + table + ';'
  
  cursor.execute(query)
  records = cursor.fetchall()

  return records


def column_stats(table, column):
  conn = psycopg2.connect(dbname='db', user='grok')
  cursor = conn.cursor()

  query = f'SELECT {column}  FROM {table};'
  cursor.execute(query)

  records = cursor.fetchall()
  array = np.array(records)

  return array.mean(), np.median(array)
